### Podstawy Sparka
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### <center>Cześć 1</center>

In [ ]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('Czesc 1 BigData').getOrCreate()

print(spark.version)

In [ ]:
data = [i + 1 for i in range(5)]

rdd = spark.sparkContext.parallelize(data)

print('RDD:', rdd.collect())

In [ ]:
df = spark.createDataFrame([('Andorra', 98), ('United Arab Emirates', 1683)], ['country_name', 'total_confirmed'])

df.show()

In [ ]:
spark.stop()

### <center>Część 2</center>

### 2.1. Wczytaj dane ze swojego pliku CSV dotyczącego danych COVID-19 (pamiętaj o nagłówku dla kolumn).

In [ ]:
import pandas as pd
import os


covid_df = pd.read_csv(os.path.join('data', 'covid.csv'))

def save_spark_df(df1, name):
    df1.write.csv(f'{name}.csv', header=True)
    df1.write.json(f'{name}.json')
    df1.write.parquet(f'{name}.parquet')

### 2.3. Wyświetl co najmniej 5 kolumn i wyjaśnij ich znaczenie dla analizy przypadków COVID-19. 

In [ ]:
covid_df = covid_df[['date', 'country_name', 'Continent', 'age_confirmed_0', 'age_confirmed_1', 'age_deceased_0', 'age_deceased_1', 'health_expenditure_usd', 'life_expectancy', 'school_closing', 'Area (km²)', 'population']]

covid_df['Area (km²)'] = covid_df['Area (km²)'].astype(float)
covid_df['population'] = covid_df['population'].astype(float)

### 2.2. Utwórz DataFrame. Pamiętaj o utworzeniu własnego schematu z wybranymi kolumnami. 

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType


spark = SparkSession.builder.appName('Czesc 2 BigData').getOrCreate()

schema = StructType([
    StructField('date', StringType(), True),
    StructField('country_name', StringType(), True),
    StructField('continent', StringType(), True),
    StructField('preschoolers_confirmed', IntegerType(), True),
    StructField('pupils_confirmed', IntegerType(), True),
    StructField('preschoolers_dead', IntegerType(), True),
    StructField('pupils_dead', IntegerType(), True),
    StructField('health_expenditure_usd', FloatType(), True),
    StructField('life_expectancy', FloatType(), True),
    StructField('school_closing', IntegerType(), True),
    StructField('area_km2', FloatType(), True),
    StructField('population', FloatType(), True)
])

spark_covid_df = spark.createDataFrame(covid_df, schema)

### 2.3. Wyświetl co najmniej 5 kolumn i wyjaśnij ich znaczenie dla analizy przypadków COVID-19.

In [ ]:
save_spark_df(spark_covid_df, name='analyze_5_columns')
spark_covid_df.show(5)

### 2.4. Dodaj kolumnę, która będzie zbierała dane, które uznasz za stosowne i które są istotne z punktu widzenia analizy COVID-19 (np. wyliczenie gęstości zaludnienia na podstawie liczby ludności i powierzchni kraju).

In [ ]:
from pyspark.sql.functions import concat, col


spark_covid_df = spark_covid_df.withColumn('population_density', spark_covid_df.population / spark_covid_df.area_km2)

save_spark_df(spark_covid_df, name='population_density')
spark_covid_df.show(5)

### 2.5. Połącz wybrane kolumny ze sobą w sposób niosący nowe informacje za pomocą funkcji concat .

In [ ]:
spark_covid_df = spark_covid_df.withColumn(
    'combined_country_continent',
    concat(col('country_name'), col('continent')),
)

save_spark_df(spark_covid_df, name='country_name_with_continent')
spark_covid_df.show(5)

### 2.6. Wyfiltruj co najmniej 10 wybranych przez siebie, istotnych informacji (np. kraj z liczbą łóżek większą od 10000 czy dzień, w którym było najmniej zachorowań w danym kraju).

### Kraje z śmiertelnością przedszkolaków większą niż 1000

In [ ]:
filtered_df = spark_covid_df.filter(col('preschoolers_dead') > 1000).distinct().select('country_name')

save_spark_df(filtered_df, name='countries_and_preschoolers_dead')
filtered_df.show(5)

### Kraje z śmierlenością uczniów większą niż 1000

In [ ]:
filtered_df = spark_covid_df.filter(col('pupils_dead') > 1000).select('country_name').distinct()

save_spark_df(filtered_df, name='countries_and_pupils_dead')
filtered_df.show(5)

### Kraje z wartością wydatków na służbę zdrowia większą niż 1000

In [ ]:
filtered_df = spark_covid_df.filter(col('health_expenditure_usd') > 1000).select('country_name').distinct()

save_spark_df(filtered_df, name='country_name_and_health_expenditure_usd')
filtered_df.show(5)

### Kraje z przewidywaną długością życia większą od 60

In [ ]:
filtered_df = spark_covid_df.filter(col('life_expectancy') > 60).select('country_name').distinct()

save_spark_df(filtered_df, name='country_name_and_life_expectancy')
filtered_df.show(5)

### Daty z liczbą potwierdzonych przypadków wśród przedszkolaków większą od 100

In [ ]:
filtered_df = spark_covid_df.filter(col('preschoolers_confirmed') > 100).select('date').distinct()

save_spark_df(filtered_df, name='date_and_preschoolers_confirmed')
filtered_df.show(5)

### Daty z liczbą potwierdzonych przypadków wśród uczniów przekraczajacą 100

In [ ]:
filtered_df = spark_covid_df.filter(col('pupils_confirmed') > 100).select('date').distinct()

save_spark_df(filtered_df, name='date_and_pupils_confirmed')
filtered_df.show(5)

### Kraje w Europie z liczbą zgonów wśród przedszkolaków większą od 20

In [ ]:
filtered_df = spark_covid_df.filter((col('continent') == 'Europe') & (col('preschoolers_dead') > 20)).select('country_name').distinct()

save_spark_df(filtered_df, name='country_name_and_continent_with_preschoolers_dead')
filtered_df.show(5)

### Kraje w Europie, których długość życia jest większa od 60 i śmiertelność przedszkolaków mniejsza od 100

In [ ]:
filtered_df = spark_covid_df.filter((col('continent') == 'Europe') & (col('preschoolers_dead') > 100) & (col('health_expenditure_usd') > 60)).select('country_name').distinct()

save_spark_df(filtered_df, name='country_name_and_continent_with_preschoolers_dead_with_health_expenditure_usd')
filtered_df.show(5)

### Kraje w Azji, których liczba śmierci przedszkolaków wynosi mniej niż 1000 i długość życia jest większa od 80

In [ ]:
filtered_df = spark_covid_df.filter((col('continent') == 'Asia') & (col('preschoolers_dead') < 1000) & (col('life_expectancy') > 80)).select('country_name').distinct()

save_spark_df(filtered_df, name='country_name_and_continent_with_preschoolers_dead_with_life_expectancy')
filtered_df.show(5)

### Kraje w Azji, w których polityka zamykania szkół była najbardziej rygorystyczna i śmiertelność uczniów była mniejsza od 1000

In [ ]:
filtered_df = spark_covid_df.filter((col('continent') == 'Asia') & (col('school_closing') == 3) & (col('pupils_dead') < 1000)).select('country_name').distinct()

save_spark_df(filtered_df, name='country_name_and_continent_with_pupils_dead')
filtered_df.show(5)

### 2.7. Wykonaj 5 agregacji na swoim DF (np. min, max, agg, count, mean) uprzednio pogrupowawszy kolumny (groupBy).

In [ ]:
aggregated_df = spark_covid_df.groupBy("country_name").agg(
    {
        'preschoolers_confirmed': 'max',
        'pupils_confirmed': 'sum',
        'preschoolers_dead': 'avg',
        'pupils_dead': 'min',
        'health_expenditure_usd': 'mean'
    }
)

save_spark_df(aggregated_df, name='aggregated_country_name')
aggregated_df.show(5)